In [52]:
from functions import * 
from IPython.display import clear_output
import os

In [53]:
# Get a handle for the record_path
record_path = '../Scripts&Records/'

In [54]:
def progress_bar(id_,array_):
    # Progress Bar
    clear_output(wait=True)
    print(f"Processing file {id_+1} of {len(array_)} files... {round(100*(id_+1)/len(array_),2)}% Complete")
    return


def status_update_msg(msg):
    clear_output(wait=True)
    print(msg)
    return

In [55]:
express_hist20 = pd.read_csv(record_path+'express_hist20.csv',index_col=[0])
pMain_hist20 = pd.read_csv(record_path+'pMain_hist20.csv',index_col=[0])

In [56]:
express_hist20.shape

(90090, 5)

In [57]:
pMain_hist20.shape

(90090, 5)

In [58]:
express_hist20['quality'].value_counts()

0    90090
Name: quality, dtype: int64

In [59]:
pMain_hist20['quality'].value_counts()

0    89457
1      633
Name: quality, dtype: int64

In [60]:
# Works as expected
def scale_cnvrt_dic(hists_df,index_of_hist_of_interest,x_or_y_axis_as_0or1):
    """
    # So, for example, using the pMain set of 20 histograms we have compiled, to convert the 0th histogram's x(as 0) axis...do the following 
    scale_cnvrt_dic(pMain_hist20,0,0)
    """
    
    # Setting up to convert the scale from the bin numbers (0-98, example only) to the dqm's histogram's scale values (-4.9 to 4.9, example only)
    tmp = hists_df[hists_df['paths']==hists_df['paths'].unique()[index_of_hist_of_interest]]
    tmp_i = np.array([(idx) for idx,i in enumerate(range(int(tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()+1)))])
    tmp_int = np.interp(tmp_i,(tmp_i.min(),tmp_i.max()),(-tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()/20,tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()/20))
    tmp_int = tmp_int.round(2)
    
    # Prepare the conversion dictionary for this histogram ['bin_coordinate':dqm_scale_value]
    dict_convertX = {}
    for idx,val in enumerate(tmp_i):
        for idxx,vall in enumerate(tmp_int):
            if idx==idxx:
                if x_or_y_axis_as_0or1 == 0:
                    dict_convertX['x_'+str(val)] = vall
                else:
                    dict_convertX['y_'+str(val)] = vall

    return dict_convertX

#Example
print('Example scale_cnvrt_dic to convert hitcoords from cern coordinates to bin coordinates (-1.8,1-.8)->(31,50)')
print(scale_cnvrt_dic(pMain_hist20,0,0))

print(f"Example hitstring: 'Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:6829.0'")

def tenths_ceil(num_str):
    """
    Currently, this gives the correct value when transforming (eta,phi)_cern coordinates to (eta,phi)_bin(the dataframe version of histograms) coordinates.
    
    This may not work in all cases, double check to make sure???
    """
    
    
    # Get num_str as string
    num_str = str(num_str)
    
    # Get num_str_fixed as num_str with the '-' removed if it exists
    num_str_fixed = num_str.replace('-','')
    
    # Create the tuples of ints and decimals
    int_tups = [('10e'+str(idx),char) for idx,char in enumerate(num_str_fixed.split(".")[0][::-1])]
    dec_tups = [('10e'+str(-1*idx-1),char) for idx,char in enumerate(num_str_fixed.split(".")[1])]
    
    # Loop through decimal tuples
    for id_,tup in enumerate(dec_tups):
        
        # Loop through each individual tuple's decimal value as it loops through tuples in decimals
        for dec in tup[1]:
            
            # if its the first decimal, skip it
            if id_ == 0:
                continue
                
            # if any other decimals are greater than 0
            if int(dec) > 0:
                
                # and if the given num_str input is negative, return the negative value truncated to the tenths
                if '-' in num_str:  
                    return -1 * ( float( ''.join( [tup[1] for tup in int_tups[::-1]] ) ) + float( '0.'+str ( int(dec_tups[0][1]) ) ) )
                
                # else if its not negative, return the postive value's ceiling to the tenths
                else:
                    return  float( ''.join( [tup[1] for tup in int_tups[::-1]] ) ) + float( '0.'+str ( int(dec_tups[0][1])+1 ) )
                
    # if none of the previous loops return a value, the num_str was not rounded up, so return the same negative value
    if '-' in num_str:
        return -1* ( float(''.join([tup[1] for tup in int_tups[::-1]]))+float('0.'+dec_tups[0][1]) )
    
    # else if none of the previous loops return a value, the num_str was not rounded up, so return the same positive value
    else:        
        return float(''.join([tup[1] for tup in int_tups[::-1]]))+float('0.'+dec_tups[0][1])

print(f'Example tenths_ceil(-1.850,1.723): ({tenths_ceil(-1.850)},{tenths_ceil(1.723)})')
    
    
def transform_hitstring(line):
    """
    Takes a line in as the format received from the txt_file which was copy/pasted from cern's dqm display on a specific histogram, then extracts the (x_hitcoord,y_hitcoord)
    """
    #line = "Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:	6829.0"
    
    color_identifier_string=line[0]
    
    hit_number = line.replace(line[0],'').split('-',1)[0]
    
    occ_val = int(line.split(':')[1].replace('\t','').split('.')[0])
    
    # Begin the process of extracting the x_hitcoord and y_hitcoord from line
    line = line.replace(color_identifier_string+hit_number+'-'+'(eta,phi)[OSRatio]=','')
    line = line.split(')')[0]
    line = line.replace("(",'')
    line = line.split(',')
    x_hitcoord, y_hitcoord = tenths_ceil( float( line[0] ) ), tenths_ceil( float ( line[1] ) )
    
    return ( x_hitcoord, y_hitcoord, color_identifier_string, hit_number, occ_val )

# Example string, works as expected
print(f'Example of transform_hitstring: {transform_hitstring("Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:	6829.0")}')

def extract_val_list(txt_file_path):
    """
        
    txt_file_path requires the pathname with the filename and file extension included.
    Example: "dir1/dir2/dir3/filename.txt"
    NOTE: Our current txt_file_path working directory is: "../hist_targets_txt_files/" such that the text file would look like "../hist_targets_txt_files/filename.txt"
    
    txt_file Structure:
    It is a .txt file structured from a paste function after copying from the dqm starting from the line that reads "NRedBins:", then "NYellowBins:", then the lines of most interest.
    From there, the lines of interest proceed line by line in a format such as the following ...
    C#-(eta,phi)[OSRatio]=(x_coord,y_coord)[7.59e+01]: occ_val 
    where
    C = color character identifier (Y for yellow bin, R for red bin)
    # = the number of the R/Y bin (if there are NYellowBins=44 then this number will be a number from 0-44)
    x_coord = the eta or x coordinate location of the point of interest (red or yellow hit)
    y_coord = the phi or y coordinate location of the point of interest (red or yellow hit)
    occ_Val = the occupancy value that was recorded at the location (x_coord,y_coord) 
    
    val_list Structure:
    It is a list of tuples whose tuples are each (x_hitcoord,y_hitcoord) such that ...
    val_list = [ (xhc_0,yhc_0), (xhc_1,yhc_1), ..., (xhc_n,yhc_n) ]
    """
    
    
    # Initialize the val_list
    val_list = []
    
       
    # Get a handle for the hist_path based on the txt_file_path ???
    hist_path = txt_file_path.replace('-','/').replace('.txt','')
    print(hist_path)
    
    # Open the text file
    # directory_path = '../hist_targets_txt_files/'
    with open('../hist_targets_txt_files/'+txt_file_path,"r") as f:       
        # Extract the path_name(first line of the txt_file) - the first line is the path_name
        path_name = f.readline()
        
        # Read the text file line by line and
        for line in f.readlines():
            
            # Skip these two lines
            if 'NRedBins' in line:
                continue
            if 'NYellowBins' in line:
                continue
            
            # Get a handle for the transform of the hitstring in each line of txt_file_path to the tuple of values of interest
            transformed_line = transform_hitstring(line)
            
            # (x_hitcoord, y_hitcoord, color_identifier_string, hit_number, occ_val, hist_path)
            val_list.append( ( transformed_line[0], transformed_line[1], transformed_line[2], transformed_line[3], transformed_line[4], hist_path ) )
    
    # return the val_list in the format as described above
    return val_list

# This example is very large and very long, turn on for debugging
# print(f'Example extract_val_list using txt_file_path="run_348251-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt:')
# display(extract_val_list("run_348251-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt"))
    
    

def prep_quality_feature(list_of_hists_df, hist_index, txt_file_path):
        
    """
    val_list_xy must be structured as follows - it is a list of tuples whose tuples are each (x_hitcoord,y_hitcoord) such that 
    val_list_xy = [ (xhc_0,yhc_0), (xhc_1,yhc_1), ..., (xhc_n,yhc_n) ]
    """
    
    
    # Convert the txt_file_path with '-'s in it to txt_file_path with '/'s in it instead
#     txt_file_path2 = txt_file_path.replace('-','/')
    
    # Extract and get a handle for val_list from txt_file_path
    val_list = extract_val_list(txt_file_path)
    
    # Convert our val_list tuple of 5 values to a tuple of 2 coordinate values (x,y)
    val_list_xy = [(tup[0],tup[1]) for tup in val_list]
    
    # If the histogram of interest is in the list of paths that are associated with specific red/yellow hit coordinates (hit_n = (x,y,color,occ,hist_path))
    print('???:',txt_file_path)
    
    if list_of_hists_df['paths'].unique()[hist_index] in [tup[5] for tup in val_list]:
        
        # Get a handle for the histogram we are constructing the quality feature for
        tmp = list_of_hists_df[list_of_hists_df['paths']==list_of_hists_df['paths'].unique()[hist_index]]
    else:
        return "Error: the unique histogram chosen as hist_index cannot be found for any of the hist_paths in val_list"
    
    # Get the coordinate conversion dictionary
    cnvrt_dic_x, cnvrt_dic_y = scale_cnvrt_dic(list_of_hists_df,hist_index,0), scale_cnvrt_dic(list_of_hists_df,hist_index,1)
    
    # Loop through the quality values, if the coordinates match the location of the hit, modify their quality value
    for idx,val in enumerate(tmp['quality'].values):
        
        # If the tuple (x,y) from histogram tmp is in the list of (x,y) tuples from the hit value list (val_list_xy)
        if (cnvrt_dic_x['x_'+str(int(tmp.iloc[idx,1]))],cnvrt_dic_y['y_'+str(int(tmp.iloc[idx,2]))]) in val_list_xy:
            # Set the quality class for this hit (0/1 for green/red, 0/1/2 for green/yellow/red ...for now we just use 0/1)
            tmp.iloc[idx,4] = 1 
    
    # Return the list_of_hists_df whose 'quality' values have been updated
    return tmp

# Example prep_quality_feature
print(f'Example prep_quality_feature for pMain_hist20 dataframe, hist_index=0, txt_file_path = ... :')
display(prep_quality_feature(pMain_hist20, 0, "run_363710-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt"))

Example scale_cnvrt_dic to convert hitcoords from cern coordinates to bin coordinates (-1.8,1-.8)->(31,50)
{'x_0': -4.9, 'x_1': -4.8, 'x_2': -4.7, 'x_3': -4.6, 'x_4': -4.5, 'x_5': -4.4, 'x_6': -4.3, 'x_7': -4.2, 'x_8': -4.1, 'x_9': -4.0, 'x_10': -3.9, 'x_11': -3.8, 'x_12': -3.7, 'x_13': -3.6, 'x_14': -3.5, 'x_15': -3.4, 'x_16': -3.3, 'x_17': -3.2, 'x_18': -3.1, 'x_19': -3.0, 'x_20': -2.9, 'x_21': -2.8, 'x_22': -2.7, 'x_23': -2.6, 'x_24': -2.5, 'x_25': -2.4, 'x_26': -2.3, 'x_27': -2.2, 'x_28': -2.1, 'x_29': -2.0, 'x_30': -1.9, 'x_31': -1.8, 'x_32': -1.7, 'x_33': -1.6, 'x_34': -1.5, 'x_35': -1.4, 'x_36': -1.3, 'x_37': -1.2, 'x_38': -1.1, 'x_39': -1.0, 'x_40': -0.9, 'x_41': -0.8, 'x_42': -0.7, 'x_43': -0.6, 'x_44': -0.5, 'x_45': -0.4, 'x_46': -0.3, 'x_47': -0.2, 'x_48': -0.1, 'x_49': 0.0, 'x_50': 0.1, 'x_51': 0.2, 'x_52': 0.3, 'x_53': 0.4, 'x_54': 0.5, 'x_55': 0.6, 'x_56': 0.7, 'x_57': 0.8, 'x_58': 0.9, 'x_59': 1.0, 'x_60': 1.1, 'x_61': 1.2, 'x_62': 1.3, 'x_63': 1.4, 'x_64': 1.5, 'x_65': 

'Error: the unique histogram chosen as hist_index cannot be found for any of the hist_paths in val_list'

In [16]:
ls ../hist_targets_txt_files/

run_348251-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt


### Main Process

1. Find the hist_index of the histogram you are interested in adding quality values to, write this run number and stream information down (find ftag if necessary if you dont remember), and find it in the dqm display to prepare the text file

In [16]:
express_hist20['paths'].unique()

array(['run_348251/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363664/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363710/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363738/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363830/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363910/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363947/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363979/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364030/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'ru

In [17]:
pMain_hist20['paths'].unique()

array(['run_363710/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363738/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363830/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363910/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363947/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363979/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364030/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364076/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364098/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'ru

2. With the run, stream, and ftag in hand, go to the web display for this particular histogram "<run_######>/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1", and copy paste the red/yellow values in the text file as was done with the first text file

3. Add the run path information(at the top of the dqm display histogram view as the first line to the text file as done previously

4. update the .txt file path in the below block, the appropriate hist_index number, and the appropriate dataframes_hist variable in the prep_quality_Feature function below.

5. concatenate as shown

6. Test to make sure 44+new values exist

In [18]:
tmp2

NameError: name 'tmp2' is not defined

In [19]:
# Get a subset that does not include unique()[hist_index] == 0
tmp = pMain_hist20[pMain_hist20['paths']!=pMain_hist20['paths'].unique()[0]]
# Prepare the modified quality values
tmp2 = prep_quality_feature(pMain_hist20, 0, "run_363710-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt")
# then concatonate the pMain_hist20 with tmp_df
pMain_hist20 = pd.concat([tmp,tmp2])

run_363710/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1
???: run_363710-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt


/cvmfs/sft.cern.ch/lcg/views/LCG_100/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [20]:
pMain_hist20['paths'].unique()

array(['run_363738/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363830/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363910/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363947/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363979/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364030/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364076/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364098/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364160/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'ru

In [29]:
pMain_hist20['quality'].value_counts()

0    89654
1      436
Name: quality, dtype: int64

In [28]:
100*pMain_hist20['quality'].value_counts().values[1]/(pMain_hist20['quality'].value_counts().values[0]+pMain_hist20['quality'].value_counts().values[1])

0.48396048396048397

SO im getting about .2% per 200 datapoint document..ouch

There should be 363 + 44 = 407. This is not the number we are getting

However, it is possible that the dqm display is not printing out all 300+ values. That is what appears to be happening.
- 200 through 363
are missing specifically
<br>

In [30]:
# Save the databases
express_hist20.to_csv(record_path+'express_hist20.csv')
pMain_hist20.to_csv(record_path+'pMain_hist20.csv')

<hr style='border:solid 1px'>

In [ ]:
from functions import * 
from IPython.display import clear_output
import os

In [72]:
# Get a handle for the record_path
record_path = '../Scripts&Records/'

def progress_bar(id_,array_):
    # Progress Bar
    clear_output(wait=True)
    print(f"Processing file {id_+1} of {len(array_)} files... {round(100*(id_+1)/len(array_),2)}% Complete")
    return


def status_update_msg(msg):
    clear_output(wait=True)
    print(msg)
    return


express_hist20 = pd.read_csv(record_path+'express_hist20.csv',index_col=[0])
pMain_hist20 = pd.read_csv(record_path+'pMain_hist20.csv',index_col=[0])


# Works as expected
def scale_cnvrt_dic(hists_df,index_of_hist_of_interest,x_or_y_axis_as_0or1):
    """
    # So, for example, using the pMain set of 20 histograms we have compiled, to convert the 0th histogram's x(as 0) axis...do the following 
    scale_cnvrt_dic(pMain_hist20,0,0)
    """
    
    # Setting up to convert the scale from the bin numbers (0-98, example only) to the dqm's histogram's scale values (-4.9 to 4.9, example only)
    tmp = hists_df[hists_df['paths']==hists_df['paths'].unique()[index_of_hist_of_interest]]
    tmp_i = np.array([(idx) for idx,i in enumerate(range(int(tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()+1)))])
    tmp_int = np.interp(tmp_i,(tmp_i.min(),tmp_i.max()),(-tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()/20,tmp[tmp.columns[x_or_y_axis_as_0or1+1]].values.max()/20))
    tmp_int = tmp_int.round(2)
    
    # Prepare the conversion dictionary for this histogram ['bin_coordinate':dqm_scale_value]
    dict_convertX = {}
    for idx,val in enumerate(tmp_i):
        for idxx,vall in enumerate(tmp_int):
            if idx==idxx:
                if x_or_y_axis_as_0or1 == 0:
                    dict_convertX['x_'+str(val)] = vall
                else:
                    dict_convertX['y_'+str(val)] = vall

    return dict_convertX

#Example
print('Example scale_cnvrt_dic to convert hitcoords from cern coordinates to bin coordinates (-1.8,1-.8)->(31,50)')
print(scale_cnvrt_dic(pMain_hist20,0,0))

print(f"Example hitstring: 'Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:6829.0'")

def tenths_ceil(num_str):
    """
    Currently, this gives the correct value when transforming (eta,phi)_cern coordinates to (eta,phi)_bin(the dataframe version of histograms) coordinates.
    
    This may not work in all cases, double check to make sure???
    """
    
    
    # Get num_str as string
    num_str = str(num_str)
    
    # Get num_str_fixed as num_str with the '-' removed if it exists
    num_str_fixed = num_str.replace('-','')
    
    # Create the tuples of ints and decimals
    int_tups = [('10e'+str(idx),char) for idx,char in enumerate(num_str_fixed.split(".")[0][::-1])]
    dec_tups = [('10e'+str(-1*idx-1),char) for idx,char in enumerate(num_str_fixed.split(".")[1])]
    
    # Loop through decimal tuples
    for id_,tup in enumerate(dec_tups):
        
        # Loop through each individual tuple's decimal value as it loops through tuples in decimals
        for dec in tup[1]:
            
            # if its the first decimal, skip it
            if id_ == 0:
                continue
                
            # if any other decimals are greater than 0
            if int(dec) > 0:
                
                # and if the given num_str input is negative, return the negative value truncated to the tenths
                if '-' in num_str:  
                    return -1 * ( float( ''.join( [tup[1] for tup in int_tups[::-1]] ) ) + float( '0.'+str ( int(dec_tups[0][1]) ) ) )
                
                # else if its not negative, return the postive value's ceiling to the tenths
                else:
                    return  float( ''.join( [tup[1] for tup in int_tups[::-1]] ) ) + float( '0.'+str ( int(dec_tups[0][1])+1 ) )
                
    # if none of the previous loops return a value, the num_str was not rounded up, so return the same negative value
    if '-' in num_str:
        return -1* ( float(''.join([tup[1] for tup in int_tups[::-1]]))+float('0.'+dec_tups[0][1]) )
    
    # else if none of the previous loops return a value, the num_str was not rounded up, so return the same positive value
    else:        
        return float(''.join([tup[1] for tup in int_tups[::-1]]))+float('0.'+dec_tups[0][1])

print(f'Example tenths_ceil(-1.850,1.723): ({tenths_ceil(-1.850)},{tenths_ceil(1.723)})')
    
    
def transform_hitstring(line):
    """
    Takes a line in as the format received from the txt_file which was copy/pasted from cern's dqm display on a specific histogram, then extracts the (x_hitcoord,y_hitcoord)
    """
    #line = "Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:	6829.0"
    
    color_identifier_string=line[0]
    
    hit_number = line.replace(line[0],'').split('-',1)[0]
    
    occ_val = int(line.split(':')[1].replace('\t','').split('.')[0])
    
    # Begin the process of extracting the x_hitcoord and y_hitcoord from line
    line = line.replace(color_identifier_string+hit_number+'-'+'(eta,phi)[OSRatio]=','')
    line = line.split(')')[0]
    line = line.replace("(",'')
    line = line.split(',')
    x_hitcoord, y_hitcoord = tenths_ceil( float( line[0] ) ), tenths_ceil( float ( line[1] ) )
    
    return ( x_hitcoord, y_hitcoord, color_identifier_string, hit_number, occ_val )

# Example string, works as expected
print(f'Example of transform_hitstring: {transform_hitstring("Y0-(eta,phi)[OSRatio]=(-1.850,1.723)[7.59e+01]:	6829.0")}')

def extract_val_list(txt_file_path):
    """
        
    txt_file_path requires the pathname with the filename and file extension included.
    Example: "dir1/dir2/dir3/filename.txt"
    NOTE: Our current txt_file_path working directory is: "../hist_targets_txt_files/" such that the text file would look like "../hist_targets_txt_files/filename.txt"
    
    txt_file Structure:
    It is a .txt file structured from a paste function after copying from the dqm starting from the line that reads "NRedBins:", then "NYellowBins:", then the lines of most interest.
    From there, the lines of interest proceed line by line in a format such as the following ...
    C#-(eta,phi)[OSRatio]=(x_coord,y_coord)[7.59e+01]: occ_val 
    where
    C = color character identifier (Y for yellow bin, R for red bin)
    # = the number of the R/Y bin (if there are NYellowBins=44 then this number will be a number from 0-44)
    x_coord = the eta or x coordinate location of the point of interest (red or yellow hit)
    y_coord = the phi or y coordinate location of the point of interest (red or yellow hit)
    occ_Val = the occupancy value that was recorded at the location (x_coord,y_coord) 
    
    val_list Structure:
    It is a list of tuples whose tuples are each (x_hitcoord,y_hitcoord) such that ...
    val_list = [ (xhc_0,yhc_0), (xhc_1,yhc_1), ..., (xhc_n,yhc_n) ]
    """
    
    
    # Initialize the val_list
    val_list = []
    
       
    # Get a handle for the hist_path based on the txt_file_path ???
    hist_path = txt_file_path.replace('-','/').replace('.txt','')
    
    # Open the text file
    # directory_path = '../hist_targets_txt_files/'
    with open('../hist_targets_txt_files/'+txt_file_path,"r") as f:       
        # Extract the path_name(first line of the txt_file) - the first line is the path_name
        path_name = f.readline()
        
        # Read the text file line by line and
        for line in f.readlines():
            
            # Skip these two lines
            if 'NRedBins' in line:
                continue
            if 'NYellowBins' in line:
                continue
            
            # Get a handle for the transform of the hitstring in each line of txt_file_path to the tuple of values of interest
            transformed_line = transform_hitstring(line)
            
            # (x_hitcoord, y_hitcoord, color_identifier_string, hit_number, occ_val, hist_path)
            val_list.append( ( transformed_line[0], transformed_line[1], transformed_line[2], transformed_line[3], transformed_line[4], hist_path ) )
    
    # return the val_list in the format as described above
    return val_list

# This example is very large and very long, turn on for debugging
# print(f'Example extract_val_list using txt_file_path="run_348251-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt:')
# display(extract_val_list("run_348251-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt"))
    
    

def prep_quality_feature(list_of_hists_df, hist_index, txt_file_path):
        
    """
    val_list_xy must be structured as follows - it is a list of tuples whose tuples are each (x_hitcoord,y_hitcoord) such that 
    val_list_xy = [ (xhc_0,yhc_0), (xhc_1,yhc_1), ..., (xhc_n,yhc_n) ]
    """
    
    
    # Convert the txt_file_path with '-'s in it to txt_file_path with '/'s in it instead
#     txt_file_path2 = txt_file_path.replace('-','/')
    
    # Extract and get a handle for val_list from txt_file_path
    val_list = extract_val_list(txt_file_path)
    
    # Convert our val_list tuple of 5 values to a tuple of 2 coordinate values (x,y)
    val_list_xy = [(tup[0],tup[1]) for tup in val_list]
    
    # If the histogram of interest is in the list of paths that are associated with specific red/yellow hit coordinates (hit_n = (x,y,color,occ,hist_path))
    if list_of_hists_df['paths'].unique()[hist_index] in [tup[5] for tup in val_list]:
        
        # Get a handle for the histogram we are constructing the quality feature for
        tmp = list_of_hists_df[list_of_hists_df['paths']==list_of_hists_df['paths'].unique()[hist_index]]
    else:
        return "Error: the unique histogram chosen as hist_index cannot be found for any of the hist_paths in val_list"
    
    # Get the coordinate conversion dictionary
    cnvrt_dic_x, cnvrt_dic_y = scale_cnvrt_dic(list_of_hists_df,hist_index,0), scale_cnvrt_dic(list_of_hists_df,hist_index,1)
    
    # Loop through the quality values, if the coordinates match the location of the hit, modify their quality value
    for idx,val in enumerate(tmp['quality'].values):
        
        # If the tuple (x,y) from histogram tmp is in the list of (x,y) tuples from the hit value list (val_list_xy)
        if (cnvrt_dic_x['x_'+str(int(tmp.iloc[idx,1]))],cnvrt_dic_y['y_'+str(int(tmp.iloc[idx,2]))]) in val_list_xy:
            # Set the quality class for this hit (0/1 for green/red, 0/1/2 for green/yellow/red ...for now we just use 0/1)
            tmp.iloc[idx,4] = 1 
    
    # Return the list_of_hists_df whose 'quality' values have been updated
    return tmp

# Example prep_quality_feature
print(f'Example prep_quality_feature for pMain_hist20 dataframe, hist_index=0, txt_file_path = ... :')
display(prep_quality_feature(pMain_hist20, 0, "run_363710-CaloMonitoring-ClusterMon-CaloCalTopoClustersNoTrigSel-2d_Rates-m_clus_etaphi_Et_thresh1.txt"))

Example scale_cnvrt_dic to convert hitcoords from cern coordinates to bin coordinates (-1.8,1-.8)->(31,50)
{'x_0': -4.9, 'x_1': -4.8, 'x_2': -4.7, 'x_3': -4.6, 'x_4': -4.5, 'x_5': -4.4, 'x_6': -4.3, 'x_7': -4.2, 'x_8': -4.1, 'x_9': -4.0, 'x_10': -3.9, 'x_11': -3.8, 'x_12': -3.7, 'x_13': -3.6, 'x_14': -3.5, 'x_15': -3.4, 'x_16': -3.3, 'x_17': -3.2, 'x_18': -3.1, 'x_19': -3.0, 'x_20': -2.9, 'x_21': -2.8, 'x_22': -2.7, 'x_23': -2.6, 'x_24': -2.5, 'x_25': -2.4, 'x_26': -2.3, 'x_27': -2.2, 'x_28': -2.1, 'x_29': -2.0, 'x_30': -1.9, 'x_31': -1.8, 'x_32': -1.7, 'x_33': -1.6, 'x_34': -1.5, 'x_35': -1.4, 'x_36': -1.3, 'x_37': -1.2, 'x_38': -1.1, 'x_39': -1.0, 'x_40': -0.9, 'x_41': -0.8, 'x_42': -0.7, 'x_43': -0.6, 'x_44': -0.5, 'x_45': -0.4, 'x_46': -0.3, 'x_47': -0.2, 'x_48': -0.1, 'x_49': 0.0, 'x_50': 0.1, 'x_51': 0.2, 'x_52': 0.3, 'x_53': 0.4, 'x_54': 0.5, 'x_55': 0.6, 'x_56': 0.7, 'x_57': 0.8, 'x_58': 0.9, 'x_59': 1.0, 'x_60': 1.1, 'x_61': 1.2, 'x_62': 1.3, 'x_63': 1.4, 'x_64': 1.5, 'x_65': 

/cvmfs/sft.cern.ch/lcg/views/LCG_100/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,paths,x,y,occ,quality
14074,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,0,0,0,0
14075,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,0,1,0,0
14076,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,0,2,0,0
14077,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,0,3,0,0
14078,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,0,4,0,0
...,...,...,...,...,...
20504,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,98,60,0,0
20505,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,98,61,0,0
20506,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,98,62,0,0
20507,run_363710/CaloMonitoring/ClusterMon/CaloCalTo...,98,63,0,0


In [121]:
# Select the id value for the histogram that we are updating the target values
hist_idx = 0

# Set the string value for the path to the text file for the histogram we are updating
txt_file_pth = ""

# Set the database we are updating the target values to
db = express_hist20

In [122]:
# Get a subset that does not include unique()[hist_index] == 0
tmp = db[db['paths']!=db['paths'].unique()[0]]
# Prepare the modified quality values
tmp2 = prep_quality_feature(db, hist_idx, txt_file_pth)
# then concatonate the pMain_hist20 with tmp_df
pMain_hist20 = pd.concat([tmp,tmp2])

# Save the databases
express_hist20.to_csv(record_path+'express_hist20.csv')
pMain_hist20.to_csv(record_path+'pMain_hist20.csv')

print("Complete.")

/cvmfs/sft.cern.ch/lcg/views/LCG_100/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Complete.


checking results...

In [132]:
express_hist20.shape

(90090, 5)

In [133]:
express_hist20['quality'].value_counts()

0    90090
Name: quality, dtype: int64

In [137]:
express_hist20['paths'].unique()

array(['run_348251/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363664/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363710/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363738/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363830/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363910/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363947/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_363979/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'run_364030/CaloMonitoring/ClusterMon/CaloCalTopoClustersNoTrigSel/2d_Rates/m_clus_etaphi_Et_thresh1',
       'ru

pMain_hist20 stats:

In [134]:
pMain_hist20.shape

(90090, 5)

In [135]:
pMain_hist20['quality'].value_counts()

0    87489
1     2601
Name: quality, dtype: int64

In [136]:
#% bad to good values
100*pMain_hist20['quality'].value_counts().values[1]/(pMain_hist20['quality'].value_counts().values[0]+pMain_hist20['quality'].value_counts().values[1])

2.8871128871128873

##### Question for Dr. Sawyer:
It appears that the express_express streams have very little errors, but it was suggested to me that express_express stream has many errors in it. It is possible that express_express contains many unknown errors and the physics_Main stream contains the same errors, but in this case they have been identified as known. On the other hand, it is possible that something is totally different between the two ...but what wouldnt make sense in this case is that the express_express then has many fewer ground_truth errors as yellow/red bins compared to the supposed to be more accurate physics_Main. What is the issue here?
- response: use either generated data as suggested. Or continue to follow through with the pMain data as suggested and get more.

##### Official solution to above(8-4-21):
According to the research I have done by the source below (1911), we can reconstruct a model similar to AnoNet and get good anomaly detection results on a single type of defect (dead cell/strip/layer) for as few as 1000 defect free images + 150 images with defect present.
We will need to acquire approx...
- 150 histograms with a single type of defect present (dead cell/strip/layer)
- 1000 histograms with no defects present (look through pMain or express_express for histograms that are clean or do not have the defect present. confounding information could still be effective as long as the binary mask only highlights a single type of defect such as dead cell/strip/layer

In [5]:
print(f"total histograms is {1000+150},\n pct histograms with a single type of defect present: {100*150/1150}%,\n pct histograms without any defects: {100*1000/1150}%")
print("\n Note: this is not the percent of bad values to good values as we previously measured. Its the percent of histograms with a defect present")

total histograms is 1150,
 pct histograms with a single type of defect present: 13.043478260869565%,
 pct histograms without any defects: 86.95652173913044%

 Note: this is not the percent of bad values to good values as we previously measured. Its the percent of histograms with a defect present


##### Use the following source to develop the CNN:
- https://arxiv.org/pdf/1911.10608.pdf
- For dilation mentioned in the paper -> https://en.wikipedia.org/wiki/Dilation_(morphology)
- the first layer is a filter initialization bank layer...to setup your own custom filter look at the custom filter section of this: https://keras.io/api/layers/initializers/
- then use the information here for how to add it to your first conv2d keras layer as described here: section 3.2 of .. https://arxiv.org/pdf/1911.10608.pdf
- equation of ellipse for dilation(maybe useful?): https://stackoverflow.com/questions/10952060/plot-ellipse-with-matplotlib-pyplot-python
- dilation over a numpy matrix: https://stackoverflow.com/questions/56735991/how-to-expand-dilate-a-numpy-array
- creating your own activation function: https://arxiv.org/pdf/1502.01852.pdf (PReLU)
- batch normalization: https://keras.io/api/layers/normalization_layers/batch_normalization/
- potentially better loss function: https://arxiv.org/pdf/1811.06861.pdf
- more on loss functions for image restorations: https://arxiv.org/pdf/1511.08861.pdf
- understanding binary cross entropy: https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a